Casear Cipher

In [99]:
#Debjyoti's modifivation - V2

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.circuit.library import CDKMRippleCarryAdder 

from qiskit.providers.basic_provider import BasicSimulator
backend = BasicSimulator()


# converting plain text to binary
# plain_text = "a"
plain_text = "10101"
# printing original string 
print("The original string is : " + str(plain_text))
 
ENCRYPTED_TEXT=''

KEY="00101" #5 in decimal
n_k=len(KEY)
print("key=",int(KEY, 2))

# Defining Quantum registers
# for i in plain_text:
# using join() + ord() + format()
# res = ''.join(format(ord(plain_text), '07b')) # for i in plain_text)
# PLAIN_TEXT=str(res)
n_p=len(plain_text)
adder = CDKMRippleCarryAdder(n_p, kind='full', name='Full Adder') # kind='full' indicates full adder
operand1 = QuantumRegister(n_p, name='key')
operand2 = QuantumRegister(n_p, name='plain_text')
anc = QuantumRegister(2, name='a') # to store carry in and carry out
cr = ClassicalRegister(n_p+1,name='cr')

circ = QuantumCircuit(operand1, operand2, anc, cr)

for i in range(len(KEY)):
    if KEY[i]=='1':
        circ.x(operand1[len(KEY)-1-i])

for i in range(len(plain_text)):
    if plain_text[i]=='1':
        circ.x(operand2[len(plain_text)-1-i])

# Note 1: CDKMRippleCarryAdder expects the qubits to have the order: carry in - 1st operand - 2nd operand - carry out. (changes will be on 2nd operator)
# Note 2: Python uses '+' operator to combine lists:

circ.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])
circ.measure(operand2[:] + [anc[1]], cr) #changes will be on operand 2 in other word (here) plain_text is changed 

# to change the key : take care of ordering the 1st operand and 2nd operand
# circ.append(adder, [anc[0]] + operand2[:] + operand1[:] + [anc[1]])
# circ.measure(operand1[:] + [anc[1]], cr) 

# print(circ)
# circ.draw("mpl")

tr_circ = transpile(circ, basis_gates = ['u3', 'cx'], optimization_level = 3)
result = backend.run(tr_circ).result()
counts = result.get_counts()

# print("encrypted text in binary:",list(counts.keys())[0])
encrypted_text = list(counts.keys())[0][1:]
print("encrypted text",encrypted_text)
# chunks = [encrypted_text[i:i+8] for i in range(0, len(encrypted_text), 8)]

# # Convert each chunk to integer and then to character
# resulting_char = ''.join([chr(int(chunk, 2)) for chunk in chunks])

# # print("encrypted char",resulting_char)
# ENCRYPTED_TEXT+=resulting_char
    
# print("ENCRYPTED_TEXT :",ENCRYPTED_TEXT)


The original string is : 10101
key= 5
encrypted text 11010


Grover search

In [100]:
plain_text

'10101'

In [101]:
cipher_text=encrypted_text
cipher_text

'11010'

In [102]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister#, BasicAer, execute, IBMQ
from qiskit.compiler import assemble
# from qiskit.tools.jupyter import *
from qiskit.visualization import *
# from ibm_quantum_widgets import CircuitComposer
# from qiskit.providers.aer import QasmSimulator, AerSimulator

# Loading your IBM Quantum account(s)
# provider = IBMQ.load_account()

In [103]:
def load_input(qc, r_key, input_binary, n):
    for i in range(n):
        if (input_binary[i] == "1"):
            qc.x(r_key[i])

In [104]:
def load_key(qc, r_key, n):
    for i in range(n):
        qc.h(r_key[i])

In [105]:
# def cipher(qc, r_key, r_text, n):
def cipher(qc, adder, anc, operand1, operand2, r_cipher, n):
    qc.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])
    # qc.measure(operand2[:] + [anc[1]], cr)
    # tr_circ = transpile(qc, basis_gates = ['u3', 'cx'], optimization_level = 3)
    # result = backend.run(tr_circ).result()
    # counts = result.get_counts()
    # e_text = list(counts.keys())[0][1:]
    # for i in e_text:
    for i in range(n):
        if cipher_text[i] == '1':
            qc.x(r_cipher[i])
        

In [106]:
def my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n):
    #Compute cipher text possibilities by XORing
    cipher(qc,adder, r_ancilla, r_key, r_text,r_cipher,n)
    
    #Checking whether the generated cipher text is equal to the given cipher text
    for i in range(n):
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
        qc.x(r_ancilla[i])
        
    #Set 'output' bit if the cipher text is matched
    qc.mcx(r_ancilla, r_output)
    
    qc.barrier()
    
    #Uncompute cipher to reset ancilla & plain text qubits
    #Reset ancilla qubit
    for i in range(n):
        qc.x(r_ancilla[i])
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
    qc.barrier()
    #Reset plain text by inverse cipher process
    cipher(qc,adder, r_ancilla, r_key, r_text,r_cipher,n)
    
    qc.barrier()
        

In [107]:
#The diffuser function is available in Qiskit
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    #Apply transformation |s> -> |00...0> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #Apply transformation |00...0> -> |11...1> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)    
    #Do multi-controlled Z gate
    qc.h(nqubits - 1)
    qc.mcx(list(range(nqubits - 1)), nqubits - 1) #multi-controlled Toffoli
    qc.h(nqubits - 1)
    #Apply transformation |11...1> -> |00...0> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    #Apply transformation |00...0> -> |s> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s
    

In [108]:
n = 5
r_text = QuantumRegister(n, 't')
r_key = QuantumRegister(n, 'k')
r_output = QuantumRegister(1, name = 'o')
r_cipher = QuantumRegister(n, 'c')
r_ancilla = QuantumRegister(n+1, 'a')
# r_ancilla = QuantumRegister(2, 'a')

r_class = ClassicalRegister(n)

qc = QuantumCircuit(r_text, r_key, r_cipher, r_ancilla, r_output, r_class)

adder = CDKMRippleCarryAdder(n, kind='full', name='Full Adder')

#loading plain text
# plain_text = '0101'
load_input(qc, r_text, plain_text, n)

#Preparing key in a superposition state
load_key(qc, r_key, n)

#loading known cipher text
# cipher_text = '1001'
load_input(qc, r_cipher, cipher_text, n)

#Preparing output qubit
qc.x(r_output)
qc.h(r_output)
qc.barrier()


from math import pi, sqrt
num_iterations = int(pi/4 * 2**(n/2.0))

# Perform the remaining iterations
for i in range(num_iterations):
    my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
    qc.barrier()
    qc.append(diffuser(n), r_key)
    qc.barrier()

qc.measure(r_key, r_class)
#qc.draw()


In [111]:
#sim = AerSimulator(method = 'matrix_product_state')
# sim = Aer.get_backend('qasm_simulator')

from qiskit.providers.basic_provider import BasicSimulator
sim = BasicSimulator()

shots = 1024
#Run and get the counts, using the matrix_product_state method
tcirc = transpile(qc, sim)
result = sim.run(tcirc, shots=1024).result()

counts = result.get_counts() #counts = result.get_counts(0)
b, c = max(counts.items(), key=lambda x: x[1])
print("Key is :", b[::-1])

Key is : 00000


In [110]:
print (result.get_counts())
print("probability =", c/shots)

{'10111': 45, '01001': 35, '11110': 40, '11101': 33, '10110': 27, '01010': 33, '00101': 33, '11001': 26, '10010': 39, '10101': 26, '00001': 36, '11010': 27, '00010': 43, '11111': 35, '01011': 29, '01110': 22, '10001': 28, '01111': 34, '00011': 25, '10000': 36, '01101': 31, '00111': 22, '01100': 39, '11100': 31, '01000': 26, '00100': 32, '00000': 38, '11000': 33, '11011': 33, '10011': 32, '10100': 15, '00110': 40}
probability = 0.0439453125


In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister#, Aer, execute
from qiskit.providers.basic_provider import BasicSimulator
sim = BasicSimulator()
# Initialize quantum and classical registers
p_bits = '0101'
q_bits = '1100'
n = len(p_bits)

p = QuantumRegister(n, 'p')
q = QuantumRegister(n, 'q')
carries = QuantumRegister(n, 'carry')
result = ClassicalRegister(n+1, 'result')  # n+1 to account for overflow carry
qc = QuantumCircuit(p, q, carries, result)

# Initialize the quantum registers with the binary numbers
for i in range(n):
    if p_bits[n-1-i] == '1':
        qc.x(p[i])  # X gate to flip qubit to 1 if the bit is 1
    if q_bits[n-1-i] == '1':
        qc.x(q[i])

# Function to perform the full adder operation using quantum gates
def full_adder(qc, a, b, c_in, sum_out, c_out):
    qc.cx(a, sum_out)
    qc.cx(b, sum_out)
    qc.ccx(a, b, c_out)
    qc.cx(sum_out, c_out)
    qc.cx(c_in, sum_out)
    qc.ccx(c_in, sum_out, c_out)

# Apply the full adder for each bit
for i in range(n):
    if i == 0:
        full_adder(qc, p[i], q[i], 0, carries[i], carries[i+1])
    else:
        full_adder(qc, p[i], q[i], carries[i], carries[i], carries[i+1])

# The last carry bit should be measured as well
qc.measure(carries[n], result[n])

# Measure the sum bits
for i in range(n):
    qc.measure(carries[i], result[i])

# Execute the circuit
tcirc = transpile(qc, sim)
result = sim.run(tcirc, shots=1024).result()
counts = result.get_counts()

# backend = Aer.get_backend('qasm_simulator')
# job = execute(qc, backend, shots=1)
# result_counts = job.result().get_counts()

# Print the result
print("Result:", counts)


CircuitError: 'duplicate qubit arguments'